# Projet IA01

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd


heart = pd.read_csv("heart.csv")


# Analyse exploratoire des données

In [ ]:
#distribution des variables catégorielles
plt.figure()
sns.histplot(x='Sex', data=heart)
plt.title('Sex distribution')
plt.show()

plt.figure()
sns.histplot(x='ChestPainType', data=heart)
plt.title('ChestPainType distribution')
plt.show()

plt.figure()
sns.histplot(x='RestingECG', data=heart)
plt.title('RestingECG distribution')
plt.show()

plt.figure()
sns.histplot(x='RestingECG', data=heart)
plt.title('RestingECG distribution')
plt.show()

plt.figure()
sns.histplot(x='ExerciseAngina', data=heart)
plt.title('ExerciseAngina distribution')
plt.show()

plt.figure()
sns.histplot(x='ST_Slope', data=heart)
plt.title('ST Slope distribution')
plt.show()


In [ ]:
#les variables catégorielles ont-elles un impact sur HeartDisease ?

matrice_corr = heart.corr(numeric_only=True)
sns.heatmap(matrice_corr, annot=True)
plt.show()

sns.countplot(x=heart['Sex'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['ChestPainType'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['RestingECG'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['ExerciseAngina'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['ST_Slope'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['FastingBS'], hue=heart['HeartDisease'])
plt.show()

In [ ]:
#identification des valeurs manquantes et aberrantes :

heart['FastingBS'] = heart['FastingBS'].astype(object)
heart['HeartDisease'] = heart['HeartDisease'].astype(object)

print("Number of Duplicates:", heart.duplicated().sum(), "\n") #nombre de doublon
print("Nombre de NaN : \n" , heart.isna().sum()) #nombre de données manquantes
    

num_df = heart.select_dtypes(include=['int64', 'float64'])

min_values = num_df.describe().loc['min']

print("Minimums des colonnes numeriques : ")
print(min_values)

cat_df = heart.select_dtypes(include=['object'])

print("Valeurs uniques des variables catégorielles : \n")
for col in cat_df.columns :
    unique_value = cat_df[col].unique()
    print(unique_value)

sns.boxplot(y=heart['Age'])
plt.title("Boxplot de l'âge")
plt.show()

sns.boxplot(y=heart['Cholesterol'])
plt.title("Boxplot du Cholesterol")
plt.show()
    
sns.boxplot(y=heart['RestingBP'])
plt.title("Boxplot de la tension")
plt.show()

sns.boxplot(y=heart['MaxHR'])
plt.title("Boxplot du maximum de fréquence cardiaque atteind")
plt.show()

sns.boxplot(y=heart['Oldpeak'])
plt.title("Boxplot de la hauteur de la dépression")
plt.show()

print("c'est bon ça marche")

### Traitons maintenant les données manquantes

In [ ]:
#Nettoyage du dataframe
# Supprimer les doublons
heart = heart.drop_duplicates()

# Remplacer les valeurs aberrantes (0 en valeurs manquantes pour les colonnes numériques)
# On peut aussi remplacer par la médianne poru éviter 
heart['Cholesterol'] = heart['Cholesterol'].replace(0, heart['Cholesterol'][heart['Cholesterol'] != 0].mean())
heart['RestingBP'] = heart['RestingBP'].replace(0, heart['RestingBP'][heart['RestingBP'] != 0].mean())


# Encoder LabelEncoder pour Sex et ExerciseAngina
from sklearn.preprocessing import LabelEncoder
le_sex = LabelEncoder()
le_angina = LabelEncoder()
heart['Sex'] = le_sex.fit_transform(heart['Sex'])
heart['ExerciseAngina'] = le_angina.fit_transform(heart['ExerciseAngina'])

# One-Hot Encoding pour ChestPainType, RestingECG et ST_Slope
heart = pd.get_dummies(heart, columns=['ChestPainType', 'RestingECG', 'ST_Slope'], drop_first=True)

print(heart.head())

### Puis les données abérrantes

##### On a décidé de supprimer les données qui sortaient des box plots

In [ ]:
# Sélection des colonnes numériques
numeric_cols = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']

# Calcul des bornes IQR
Q1 = heart[numeric_cols].quantile(0.25)
Q3 = heart[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identification des outliers
outliers_mask = (heart[numeric_cols] < lower_bound) | (heart[numeric_cols] > upper_bound)

# Résultats
print("Nombre d'outliers par variable (après imputation des 0) :")
print(outliers_mask.sum())

total_outlier_rows = outliers_mask.any(axis=1).sum()
total_rows = len(heart)
percent = (total_outlier_rows / total_rows) * 100


# Affichage des bornes
print("\nBornes utilisées :")
print(pd.DataFrame({'Lower': lower_bound, 'Upper': upper_bound}))

##### En cherchant un petit peu et en annalysant plus en profondeur ces données, on remarque que les outliers détectés par les boîtes à moustache ne sont pas très pertinents. En effet, tous sont très significatif sur l'état du patient. De plus en cherchant sur internet, ces données ne sont pas impossibles mais témoignent justement d'un serieux problème.

In [ ]:

for i in numeric_cols: 
    Q1 = heart[i].quantile(0.25)
    Q3 = heart[i].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    if i == 'MaxHR':
        print(heart[(heart[i] < lower) | (heart[i] > upper)][[i,'HeartDisease']].sort_values(by=i,ascending=True).head(5))
    else:
        print(heart[(heart[i] < lower) | (heart[i] > upper)][[i,'HeartDisease']].sort_values(by=i, ascending=False).head(5))

##### On voit clairement ici que ces valeurs identifiées comme abérrantes ont un sens puisqu'elles traduisent presques toutes une maladie.

## Train/Test split + standardisation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Y = heart['HeartDisease']
X = heart.drop(['HeartDisease'], axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=42)

# Centrées réduites de notre dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## KNN

In [ ]:
heart.head(5)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# Tester plusieurs valeurs de k
k_values = range(1, 21)
accuracies_test_man = []
accuracies_train_man = []

accuracies_test_euc=[]
accuracies_train_euc=[]

for k in k_values:
    knn_classifier = KNeighborsClassifier(n_neighbors=k, p=1, metric='minkowski')
    knn_classifier.fit(X_train_scaled, Y_train)
    y_pred_test = knn_classifier.predict(X_test_scaled)
    y_pred_train = knn_classifier.predict(X_train_scaled)
    
    accuracy = np.sum(y_pred_test == Y_test) / len(Y_test)
    accuracies_test_man.append(accuracy)
    accuracy = np.sum(y_pred_train == Y_train) / len(Y_train)
    accuracies_train_man.append(accuracy)

    knn_classifier = KNeighborsClassifier(n_neighbors=k, p=2, metric='minkowski')
    knn_classifier.fit(X_train_scaled, Y_train)
    y_pred_test = knn_classifier.predict(X_test_scaled)
    y_pred_train = knn_classifier.predict(X_train_scaled)
    
    accuracy = np.sum(y_pred_test == Y_test) / len(Y_test)
    accuracies_test_euc.append(accuracy)
    accuracy = np.sum(y_pred_train == Y_train) / len(Y_train)
    accuracies_train_euc.append(accuracy)

# Visualiser la précision en fonction de k
plt.plot(k_values, accuracies_test_man, marker='x', label = 'Test_man')
plt.plot(k_values, accuracies_train_man, marker='x', label = 'Train_man')
plt.plot(k_values, accuracies_train_euc, marker='x', label = 'Train_euc')
plt.plot(k_values, accuracies_test_euc, marker='x', label = 'Test_euc')
plt.title('Exactitude du modèle en fonction de k')
plt.xlabel('k')
plt.ylabel('Exactitude')
plt.xticks(k_values)
plt.legend()
plt.grid()
plt.show()

In [ ]:
print("En tulisant les k=9 plus proches voisins...\n")
knn_classifier = KNeighborsClassifier(n_neighbors=9, p=1, metric='minkowski')
knn_classifier.fit(X_train_scaled, Y_train)
y_pred_test = knn_classifier.predict(X_test_scaled)
y_pred_train = knn_classifier.predict(X_train_scaled)
print(f"L'éxactitude du modèle en utilisant la distance de Manhattan est de \n{accuracy_score(Y_test, y_pred_test):.4f} pour le test \n{accuracy_score(Y_train, y_pred_train):.4f} pour le train")
knn_classifier = KNeighborsClassifier(n_neighbors=9, p=2, metric='minkowski')
knn_classifier.fit(X_train_scaled, Y_train)
y_pred_test = knn_classifier.predict(X_test_scaled)
y_pred_train = knn_classifier.predict(X_train_scaled)
print(f"L'éxactitude du modèle en utilisant la distance euclidienne est de \n{accuracy_score(Y_test, y_pred_test):.4f} pour le test \n{accuracy_score(Y_train, y_pred_train):.4f} pour le train")
